In [1]:
from datasets import load_dataset

# path = "https://drive.usercontent.google.com/download?id=11EqyBH_hV35g44GVeLSyWpgVLy59J2QC&export=download&authuser=0"
# korbi_ds = load_dataset("webdataset", data_files={"train": path}, split="train")

In [2]:
# !gdown --id 11EqyBH_hV35g44GVeLSyWpgVLy59J2QC

In [1]:
# import pandas as pd
# import tarfile


# tar_path = '../data/echr-embeddings.tar.gz'

# # Open the tar.gz file
# with tarfile.open(tar_path, 'r:gz') as tar:
#     tar.extractall()




In [ ]:
# !pip install chromadb

In [ ]:
import chromadb
client = chromadb.PersistentClient(path="echr-qa-experiments/data/chroma_gtr_t5_xl_db")

In [ ]:
collections = client.list_collections()
print("Collections in the database:")
for collection in collections:
    print(f"- {collection.name}")
    

In [3]:
collection_name = collections[0].name  # Or specify the collection name directly
collection = client.get_collection(name=collection_name)

# Retrieve all items in the collection (be cautious if the collection is large)
results = collection.get()

In [ ]:
results.keys()

In [ ]:
import pandas as pd

# Prepare data for the DataFrame
data = {
    'id': results['ids'],
    'document': results.get('documents', [None] * len(results['ids'])),
    'embedding': results.get('embeddings', [None] * len(results['ids'])),
    'metadata': results.get('metadatas', [None] * len(results['ids'])),
    
}

df = pd.DataFrame(data)
df.head(1)

In [ ]:
df.head(10)

In [7]:
from tqdm import tqdm
import json
import pandas as pd
import re

metadata_df = pd.json_normalize(df['metadata'])
df = pd.concat([df.drop('metadata', axis=1), metadata_df], axis=1)

In [8]:
df.sort_values(by=['case_id', 'paragraph_number'], inplace=True)

cases = []
grouped = df.groupby('case_id')

In [ ]:
for case_id, group in tqdm(grouped):
    case_name = group['case_name'].iloc[0]
    paragraphs = []
    
    for i, (_, row) in enumerate(group.iterrows()):
        print(row['document'])
        if i == 20:
            break
    break

In [ ]:
for case_id, group in tqdm(grouped):
    case_name = group['case_name'].iloc[0]
    paragraphs = []
    
    for i, (_, row) in enumerate(group.iterrows()):
        document = row['document']
        if i == 0:
            paragraphs.append(document)
        else:
            pattern = r'^.*?§\s*'
            document_cleaned = re.sub('^' + pattern, '', document)
            paragraphs.append(document_cleaned)
    cases.append({'case_id': case_id, 'paragraphs': paragraphs})

In [11]:
# Save the results to a JSONL file
with open('cases_paragraphs.jsonl', 'w', encoding='utf-8') as f:
    for case in cases:
        f.write(json.dumps(case) + '\n')

In [ ]:
cases[0]